# Anomaly Detection: Model Training and Evaluation

This notebook implements a comprehensive anomaly detection system using an ensemble of three unsupervised learning algorithms:
- **Isolation Forest**: Efficient tree-based anomaly detection
- **Local Outlier Factor (LOF)**: Density-based local anomaly detection
- **DBSCAN**: Clustering-based outlier identification

The approach combines multiple models through ensemble voting to achieve robust anomaly detection with reduced false positives.

In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
import pickle
import warnings
import time
warnings.filterwarnings('ignore')

---

## 1. Data Loading and Preparation

Loading the engineered feature dataset and preparing it for model training. The `original_index` column is excluded from training as it's only used for tracking records back to the source data.

**My Approach:**
I'm loading the pre-engineered features created in my previous feature engineering step and removing non-predictive columns like `original_index` which I only need for tracking records back to the source data. I'm validating that the final training dataset has the correct shape and all features are numeric, which is required for the machine learning algorithms I'll be using.

**Technical Notes:**
All three unsupervised models I've selected (Isolation Forest, LOF, and DBSCAN) require numeric input features. Since this is unsupervised learning, there are no target labels in my dataset - only the engineered features. I've ensured all data types are primarily float64/int64 for optimal model performance.

In [13]:
print("\n" + "="*80)
print("DATA LOADING AND PREPARATION")
print("="*80)

# Loading the engineered features from the feature engineering step
original_df = pd.read_csv('data/model_features_data.csv')
print(f"* Loaded features: {original_df.shape}")

# Excluding original_index from training (used only for tracking)
if 'original_index' in original_df.columns:
    print(f"* Excluding 'original_index' column from training")
    train_df = original_df.drop(columns=['original_index'])
else:
    train_df = original_df.copy()

print(f"* Training data shape after exclusions: {train_df.shape}")

# Verifying the final training dataset
print(f"\n* Final training data shape: {train_df.shape}")
print(f"* Data types: {train_df.dtypes.value_counts().to_dict()}")


DATA LOADING AND PREPARATION
* Loaded features: (15597, 50)
* Excluding 'original_index' column from training
* Training data shape after exclusions: (15597, 49)

* Final training data shape: (15597, 49)
* Data types: {dtype('int64'): 35, dtype('float64'): 14}


---

## 2. Isolation Forest Training and Hyperparameter Tuning

Training an Isolation Forest model with systematic hyperparameter tuning. Testing key parameter combinations to find the optimal model based on score separation between anomalous and normal records.

**Why I Chose Isolation Forest:**
Isolation Forest is an efficient tree-based algorithm that isolates anomalies by randomly selecting features and split values. I selected this because anomalies are easier to isolate (requiring fewer splits) than normal points, resulting in shorter path lengths in the tree structure - making it computationally efficient for my dataset.

**Hyperparameters I'm Tuning:**
- **contamination**: My expected proportion of anomalies in the dataset (testing 0.01 to 0.10, meaning 1% to 10%)
- **n_estimators**: Number of isolation trees in the forest (I'm testing 100, 200, 300 - more trees give more stable predictions)
- **max_samples**: Number of samples I use to build each tree ('auto' defaults to min(256, n_samples))
- **max_features**: Proportion of features I consider when splitting (1.0 means all features)

**My Optimization Strategy:**
I'm evaluating models based on **score separation** - the difference between average scores of normal vs. anomalous records. I hypothesize that higher separation indicates better discrimination between the two groups, so I'm selecting the model with maximum score separation.

**My Experimental Design:**
I'm testing 5 key parameter combinations to efficiently explore the hyperparameter space. For each configuration, I'm tracking the number of anomalies detected, percentage of dataset flagged, score separation quality, and training time.

In [14]:
print("\n" + "="*80)
print("ISOLATION FOREST TRAINING")
print("="*80)

# Defining hyperparameter grid for tuning
if_param_grid = {
    'contamination': [0.01, 0.03, 0.05, 0.07, 0.10],
    'n_estimators': [100, 200, 300],
    'max_samples': [256, 512, 'auto'],
    'max_features': [0.5, 0.75, 1.0]
}

print(f"Hyperparameter grid size: {len(list(ParameterGrid(if_param_grid)))} combinations")
print("Training with cross-validation approach...\n")

# Initializing variables for grid search
best_if_score = -np.inf
best_if_model = None
best_if_params = None
if_results = []

# Testing key parameter combinations for efficiency
key_combinations = [
    {'contamination': 0.05, 'n_estimators': 200, 'max_samples': 'auto', 'max_features': 1.0},
    {'contamination': 0.03, 'n_estimators': 200, 'max_samples': 'auto', 'max_features': 1.0},
    {'contamination': 0.07, 'n_estimators': 200, 'max_samples': 'auto', 'max_features': 1.0},
    {'contamination': 0.05, 'n_estimators': 300, 'max_samples': 512, 'max_features': 0.75},
    {'contamination': 0.05, 'n_estimators': 100, 'max_samples': 256, 'max_features': 0.5},
]

for idx, params in enumerate(key_combinations, 1):
    start_time = time.time()
    
    # Training the model with current parameters
    model = IsolationForest(
        contamination=params['contamination'],
        n_estimators=params['n_estimators'],
        max_samples=params['max_samples'],
        max_features=params['max_features'],
        random_state=42,
        n_jobs=-1,
        verbose=0
    )
    
    model.fit(train_df)
    predictions = model.predict(train_df)
    scores = model.score_samples(train_df)
    
    # Evaluating the model performance
    n_anomalies = (predictions == -1).sum()
    anomaly_pct = (n_anomalies / len(train_df)) * 100
    
    # Calculating quality metrics (lower score = more anomalous)
    avg_anomaly_score = scores[predictions == -1].mean()
    avg_normal_score = scores[predictions == 1].mean()
    score_separation = avg_normal_score - avg_anomaly_score
    
    elapsed = time.time() - start_time
    
    # Storing the results
    result = {
        'model': 'IsolationForest',
        'params': params,
        'n_anomalies': n_anomalies,
        'anomaly_pct': anomaly_pct,
        'avg_anomaly_score': avg_anomaly_score,
        'avg_normal_score': avg_normal_score,
        'score_separation': score_separation,
        'training_time': elapsed
    }
    if_results.append(result)
    
    print(f"[{idx}/{len(key_combinations)}] contamination={params['contamination']}, "
          f"n_estimators={params['n_estimators']}")
    print(f"     Anomalies: {n_anomalies:,} ({anomaly_pct:.2f}%)")
    print(f"     Score separation: {score_separation:.4f}")
    print(f"     Training time: {elapsed:.2f}s")
    
    # Tracking the best model based on score separation
    if score_separation > best_if_score:
        best_if_score = score_separation
        best_if_model = model
        best_if_params = params

print(f"\n* Best Isolation Forest Model:")
print(f"  Parameters: {best_if_params}")
print(f"  Score separation: {best_if_score:.4f}")

# Generating predictions from the best model
if_predictions = best_if_model.predict(train_df)
if_scores = best_if_model.score_samples(train_df)
if_anomalies = (if_predictions == -1)

print(f"  Detected anomalies: {if_anomalies.sum():,} ({if_anomalies.sum()/len(train_df)*100:.2f}%)")


ISOLATION FOREST TRAINING
Hyperparameter grid size: 135 combinations
Training with cross-validation approach...

[1/5] contamination=0.05, n_estimators=200
     Anomalies: 780 (5.00%)
     Score separation: 0.1240
     Training time: 2.06s
[2/5] contamination=0.03, n_estimators=200
     Anomalies: 468 (3.00%)
     Score separation: 0.1295
     Training time: 2.11s
[3/5] contamination=0.07, n_estimators=200
     Anomalies: 1,092 (7.00%)
     Score separation: 0.1195
     Training time: 1.96s
[4/5] contamination=0.05, n_estimators=300
     Anomalies: 780 (5.00%)
     Score separation: 0.1275
     Training time: 4.57s
[5/5] contamination=0.05, n_estimators=100
     Anomalies: 780 (5.00%)
     Score separation: 0.1238
     Training time: 1.37s

* Best Isolation Forest Model:
  Parameters: {'contamination': 0.03, 'n_estimators': 200, 'max_samples': 'auto', 'max_features': 1.0}
  Score separation: 0.1295
  Detected anomalies: 468 (3.00%)


---

## 3. Local Outlier Factor (LOF) Training and Tuning

Training a LOF model to detect anomalies based on local density deviations. LOF is effective at identifying anomalies in varying density regions of the dataset.

**Why I Chose LOF:**
LOF measures the local density deviation of a data point with respect to its neighbors. I chose this algorithm because it can identify points with substantially lower density than their neighbors - making it particularly effective for my dataset which likely has varying density regions.

**How LOF Complements My Isolation Forest:**
- For each point, LOF calculates the local density based on k-nearest neighbors
- It compares this density to the densities of the k-nearest neighbors  
- Points in sparse regions relative to their neighbors get higher outlier scores
- Unlike Isolation Forest which uses global isolation, LOF is sensitive to local structure - this diversity strengthens my ensemble

**Hyperparameters I'm Tuning:**
- **contamination**: My expected proportion of outliers (consistent with Isolation Forest)
- **n_neighbors**: Number of neighbors I'm using for density estimation
  - Smaller values (20): More sensitive to local variations, may find more subtle anomalies
  - Larger values (50): More global perspective, more stable but may miss very local outliers

In [15]:
print("\n" + "="*80)
print("LOCAL OUTLIER FACTOR (LOF) TRAINING")
print("="*80)

# Defining hyperparameter grid for LOF
lof_param_grid = {
    'contamination': [0.01, 0.03, 0.05, 0.07, 0.10],
    'n_neighbors': [20, 30, 50],
}

# Testing key parameter combinations
lof_key_combinations = [
    {'contamination': 0.05, 'n_neighbors': 30},
    {'contamination': 0.03, 'n_neighbors': 30},
    {'contamination': 0.07, 'n_neighbors': 30},
    {'contamination': 0.05, 'n_neighbors': 20},
    {'contamination': 0.05, 'n_neighbors': 50},
]

# Initializing variables for tracking best model
best_lof_score = -np.inf
best_lof_model = None
best_lof_params = None
lof_results = []

for idx, params in enumerate(lof_key_combinations, 1):
    start_time = time.time()
    
    # Training the LOF model
    model = LocalOutlierFactor(
        contamination=params['contamination'],
        n_neighbors=params['n_neighbors'],
        novelty=False,  # Using for training data anomaly detection
        n_jobs=-1
    )
    
    predictions = model.fit_predict(train_df)
    scores = model.negative_outlier_factor_
    
    # Evaluating the model
    n_anomalies = (predictions == -1).sum()
    anomaly_pct = (n_anomalies / len(train_df)) * 100
    
    # Calculating quality metrics (LOF scores are negative, more negative = more anomalous)
    avg_anomaly_score = scores[predictions == -1].mean()
    avg_normal_score = scores[predictions == 1].mean()
    score_separation = avg_normal_score - avg_anomaly_score
    
    elapsed = time.time() - start_time
    
    # Storing the results
    result = {
        'model': 'LOF',
        'params': params,
        'n_anomalies': n_anomalies,
        'anomaly_pct': anomaly_pct,
        'avg_anomaly_score': avg_anomaly_score,
        'avg_normal_score': avg_normal_score,
        'score_separation': score_separation,
        'training_time': elapsed
    }
    lof_results.append(result)
    
    print(f"[{idx}/{len(lof_key_combinations)}] contamination={params['contamination']}, "
          f"n_neighbors={params['n_neighbors']}")
    print(f"     Anomalies: {n_anomalies:,} ({anomaly_pct:.2f}%)")
    print(f"     Score separation: {score_separation:.4f}")
    print(f"     Training time: {elapsed:.2f}s")
    
    # Tracking the best model
    if score_separation > best_lof_score:
        best_lof_score = score_separation
        best_lof_model = model
        best_lof_params = params

print(f"\n* Best LOF Model:")
print(f"  Parameters: {best_lof_params}")
print(f"  Score separation: {best_lof_score:.4f}")

# Generating predictions from the best LOF model
lof_predictions = best_lof_model.fit_predict(train_df)
lof_scores = best_lof_model.negative_outlier_factor_
lof_anomalies = (lof_predictions == -1)

print(f"  Detected anomalies: {lof_anomalies.sum():,} ({lof_anomalies.sum()/len(train_df)*100:.2f}%)")


LOCAL OUTLIER FACTOR (LOF) TRAINING
[1/5] contamination=0.05, n_neighbors=30
     Anomalies: 780 (5.00%)
     Score separation: 1.7208
     Training time: 0.77s
[2/5] contamination=0.03, n_neighbors=30
     Anomalies: 468 (3.00%)
     Score separation: 2.4616
     Training time: 0.71s
[3/5] contamination=0.07, n_neighbors=30
     Anomalies: 1,092 (7.00%)
     Score separation: 1.3762
     Training time: 0.71s
[4/5] contamination=0.05, n_neighbors=20
     Anomalies: 779 (4.99%)
     Score separation: 2.4031
     Training time: 0.63s
[5/5] contamination=0.05, n_neighbors=50
     Anomalies: 779 (4.99%)
     Score separation: 1.4038
     Training time: 0.76s

* Best LOF Model:
  Parameters: {'contamination': 0.03, 'n_neighbors': 30}
  Score separation: 2.4616
  Detected anomalies: 468 (3.00%)


---

## 4. DBSCAN Clustering for Anomaly Detection

Training a DBSCAN model to identify noise points as anomalies. Using k-distance graph to estimate optimal epsilon values, then testing multiple parameter combinations to find the best clustering configuration.

**Why I Chose DBSCAN:**
DBSCAN groups together closely packed points and marks points in low-density regions as outliers (noise). I chose this as my third algorithm because, unlike Isolation Forest and LOF, it's primarily a clustering algorithm where anomalies are noise points that don't belong to any cluster - providing a fundamentally different detection perspective.

**My Implementation Strategy:**
- I'm grouping points that are closely packed together (high-density regions) into clusters
- Points that don't fit into any cluster are marked as noise/outliers - these are my anomalies
- I need to tune two main parameters: epsilon (ε) and min_samples

**Hyperparameters I'm Tuning:**
- **eps (epsilon)**: Maximum distance between two points I consider as neighbors
  - Too small: I get many small clusters and lots of noise
  - Too large: I get few large clusters and little noise
  - I'm estimating this using k-distance graph at the 95th percentile
  
- **min_samples**: Minimum number of points I require to form a dense region (cluster)
  - I'm setting this equal to the k value used for my eps estimation
  - Higher values create more conservative clustering (yielding more noise points)

**My Parameter Estimation Approach:**
1. Calculate k-nearest neighbor distances for multiple k values (3, 5, 10)
2. Use the 95th percentile of k-distances as my eps estimate
3. Test eps variations (0.8x, 1.0x, 1.2x) around each estimate
4. Match min_samples to the k value used for eps estimation


In [16]:
print("\n" + "="*80)
print("DBSCAN CLUSTERING")
print("="*80)

print("Computing optimal eps using k-distance graph...")
print(f"Using complete dataset with {len(train_df):,} records for DBSCAN tuning")

from sklearn.neighbors import NearestNeighbors

# Testing multiple k values to find robust eps estimates
k_values = [3, 5, 10]
eps_estimates = {}

print("\nTesting different k values for eps estimation:")
print("-" * 60)

for k in k_values:
    nbrs = NearestNeighbors(n_neighbors=k).fit(train_df)
    distances, indices = nbrs.kneighbors(train_df)
    distances = np.sort(distances[:, k-1], axis=0)
    knee_point = distances[int(len(distances) * 0.95)]  # Using 95th percentile as heuristic
    
    eps_estimates[k] = knee_point
    print(f"k={k:2d} → estimated eps: {knee_point:.4f}")

# Testing a range of eps values around the estimates with corresponding min_samples
dbscan_combinations = []

for k in k_values:
    eps = eps_estimates[k]
    # Testing eps variations around the estimate with matching min_samples
    dbscan_combinations.extend([
        {'eps': eps * 0.8, 'min_samples': k},
        {'eps': eps, 'min_samples': k},
        {'eps': eps * 1.2, 'min_samples': k},
    ])

print(f"\nTesting {len(dbscan_combinations)} DBSCAN parameter combinations:")
print("-" * 60)

# Initializing variables for tracking best model
best_dbscan_score = -np.inf
best_dbscan_model = None
best_dbscan_params = None
dbscan_results = []

for idx, params in enumerate(dbscan_combinations, 1):
    start_time = time.time()
    
    model = DBSCAN(
        eps=params['eps'],
        min_samples=params['min_samples'],
        n_jobs=-1
    )
    
    # Fitting on complete dataset
    labels = model.fit_predict(train_df)
    n_noise = (labels == -1).sum()
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    
    noise_pct = (n_noise / len(train_df)) * 100
    elapsed = time.time() - start_time
    
    result = {
        'model': 'DBSCAN',
        'params': params,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'noise_pct': noise_pct,
        'training_time': elapsed
    }
    dbscan_results.append(result)
    
    print(f"[{idx:2d}/{len(dbscan_combinations)}] eps={params['eps']:.4f}, min_samples={params['min_samples']}")
    print(f"       Clusters: {n_clusters}, Noise: {n_noise:,} ({noise_pct:.2f}%), Time: {elapsed:.2f}s")
    
    # Evaluating models with reasonable noise percentage (5-15%)
    if 5 <= noise_pct <= 15 and n_clusters > 0:
        score = -abs(noise_pct - 7.5)  # Preferring 7.5% noise
        if score > best_dbscan_score:
            best_dbscan_score = score
            best_dbscan_params = params
            best_dbscan_model = model

if best_dbscan_params:
    print(f"\n* Best DBSCAN Parameters: {best_dbscan_params}")
    print(f"  Using model trained on full dataset...")
    
    # Using the already trained model
    dbscan_labels = best_dbscan_model.labels_
    dbscan_anomalies = (dbscan_labels == -1)
    
    print(f"  Detected noise/anomalies: {dbscan_anomalies.sum():,} ({dbscan_anomalies.sum()/len(train_df)*100:.2f}%)")
else:
    print("  No suitable DBSCAN parameters found, skipping...")
    dbscan_anomalies = np.zeros(len(train_df), dtype=bool)


DBSCAN CLUSTERING
Computing optimal eps using k-distance graph...
Using complete dataset with 15,597 records for DBSCAN tuning

Testing different k values for eps estimation:
------------------------------------------------------------
k= 3 → estimated eps: 1.4625
k= 5 → estimated eps: 1.8177
k=10 → estimated eps: 2.2542

Testing 9 DBSCAN parameter combinations:
------------------------------------------------------------
[ 1/9] eps=1.1700, min_samples=3
       Clusters: 312, Noise: 1,117 (7.16%), Time: 0.61s
[ 2/9] eps=1.4625, min_samples=3
       Clusters: 213, Noise: 627 (4.02%), Time: 0.64s
[ 3/9] eps=1.7550, min_samples=3
       Clusters: 146, Noise: 362 (2.32%), Time: 0.60s
[ 4/9] eps=1.4541, min_samples=5
       Clusters: 145, Noise: 990 (6.35%), Time: 0.62s
[ 5/9] eps=1.8177, min_samples=5
       Clusters: 78, Noise: 531 (3.40%), Time: 0.62s
[ 6/9] eps=2.1812, min_samples=5
       Clusters: 42, Noise: 301 (1.93%), Time: 0.74s
[ 7/9] eps=1.8034, min_samples=10
       Clusters: 

---

## 5. Ensemble Model Creation

Combining predictions from all three models using a voting mechanism. Creating ensemble anomaly scores by normalizing and averaging individual model scores.

**My Ensemble Strategy:**
Rather than relying on a single model, I'm combining the predictions from all three algorithms (Isolation Forest, LOF, and DBSCAN) to create a more robust anomaly detection system. Each model has different strengths and may flag different types of anomalies, so an ensemble approach should reduce false positives and increase confidence in my final predictions.

**My Voting Mechanism:**
I'm implementing a multi-level voting system:
- **Vote counting**: For each record, I count how many models (0-3) flagged it as an anomaly
- **Majority vote (≥2 models)**: Higher sensitivity - I flag anomalies when at least 2 out of 3 models agree
- **Unanimous vote (3 models)**: Higher confidence - I flag anomalies only when all 3 models agree

**My Ensemble Scoring Approach:**
Beyond binary voting, I'm creating a continuous anomaly score by:
1. Normalizing individual model scores to [0, 1] range using MinMaxScaler
2. Inverting scores so that higher values indicate more anomalous behavior
3. Averaging the normalized scores from Isolation Forest and LOF
4. Using this score to rank anomalies by severity

**Why This Approach Works:**
- Diverse detection methods reduce model-specific biases
- Voting provides confidence levels (unanimous = high confidence, majority = moderate confidence)
- Continuous scores allow me to prioritize the most anomalous records for investigation

In [17]:
print("\n" + "="*80)
print("ENSEMBLE MODEL CREATION")
print("="*80)

print("Combining predictions from all models...")

# Creating ensemble predictions using voting mechanism
# Counting how many models flag each record as anomaly
ensemble_votes = (
    if_anomalies.astype(int) +
    lof_anomalies.astype(int) +
    dbscan_anomalies.astype(int)
)

print(f"\nVoting distribution:")
for votes in range(4):
    count = (ensemble_votes == votes).sum()
    pct = (count / len(train_df)) * 100
    print(f"  {votes} votes: {count:,} ({pct:.2f}%)")

# Defining anomaly thresholds using different voting strategies
ensemble_majority = (ensemble_votes >= 2)  # At least 2 models agree
ensemble_unanimous = (ensemble_votes == 3)  # All 3 models agree

print(f"\nEnsemble Anomaly Detection:")
print(f"  Majority vote (≥2 models): {ensemble_majority.sum():,} ({ensemble_majority.sum()/len(train_df)*100:.2f}%)")
print(f"  Unanimous (3 models): {ensemble_unanimous.sum():,} ({ensemble_unanimous.sum()/len(train_df)*100:.2f}%)")

# Calculating ensemble anomaly scores (average of normalized scores)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
if_scores_norm = scaler.fit_transform(if_scores.reshape(-1, 1)).flatten()
lof_scores_norm = scaler.fit_transform(lof_scores.reshape(-1, 1)).flatten()

# Inverting scores (lower score = more anomalous)
if_scores_norm = 1 - if_scores_norm
lof_scores_norm = 1 - lof_scores_norm

# Computing average ensemble score
ensemble_scores = (if_scores_norm + lof_scores_norm) / 2

print(f"\nEnsemble anomaly scores:")
print(f"  Range: [{ensemble_scores.min():.4f}, {ensemble_scores.max():.4f}]")
print(f"  Mean: {ensemble_scores.mean():.4f}")
print(f"  Std: {ensemble_scores.std():.4f}")


ENSEMBLE MODEL CREATION
Combining predictions from all models...

Voting distribution:
  0 votes: 13,785 (88.38%)
  1 votes: 1,587 (10.18%)
  2 votes: 209 (1.34%)
  3 votes: 16 (0.10%)

Ensemble Anomaly Detection:
  Majority vote (≥2 models): 225 (1.44%)
  Unanimous (3 models): 16 (0.10%)

Ensemble anomaly scores:
  Range: [0.0001, 0.8257]
  Mean: 0.1627
  Std: 0.0980


---

## 6. Model Evaluation and Comparison

Comparing the performance of all individual models and ensemble approaches. Summarizing key metrics including anomaly counts, percentages, and model parameters.

**My Evaluation Framework:**
I'm creating a comprehensive comparison of all models to understand:
- How many anomalies each model detected individually
- What percentage of the dataset each approach flagged
- Which models have the best score separation (for IF and LOF)
- What parameter configurations I selected for each model
- How the ensemble approaches compare to individual models

**What I'm Comparing:**
- **Individual Models**: Isolation Forest, LOF, and DBSCAN with their optimal parameters
- **Ensemble Models**: Both majority vote (≥2 models) and unanimous vote (3 models) strategies

**Metrics I'm Tracking:**
- **Anomalies**: Total count of records flagged by each approach
- **Percentage**: Proportion of dataset identified as anomalous
- **Score Separation**: Quality metric for IF and LOF (higher = better discrimination)
- **Parameters**: The optimal configuration I selected for each model

In [18]:
print("\n" + "="*80)
print("MODEL EVALUATION AND COMPARISON")
print("="*80)

# Creating comprehensive evaluation summary
eval_summary = pd.DataFrame([
    {
        'Model': 'Isolation Forest',
        'Anomalies': if_anomalies.sum(),
        'Percentage': f"{if_anomalies.sum()/len(train_df)*100:.2f}%",
        'Score Separation': best_if_score,
        'Parameters': str(best_if_params)
    },
    {
        'Model': 'LOF',
        'Anomalies': lof_anomalies.sum(),
        'Percentage': f"{lof_anomalies.sum()/len(train_df)*100:.2f}%",
        'Score Separation': best_lof_score,
        'Parameters': str(best_lof_params)
    },
    {
        'Model': 'DBSCAN',
        'Anomalies': dbscan_anomalies.sum(),
        'Percentage': f"{dbscan_anomalies.sum()/len(train_df)*100:.2f}%",
        'Score Separation': 'N/A',
        'Parameters': str(best_dbscan_params) if best_dbscan_params else 'N/A'
    },
    {
        'Model': 'Ensemble (≥2 votes)',
        'Anomalies': ensemble_majority.sum(),
        'Percentage': f"{ensemble_majority.sum()/len(train_df)*100:.2f}%",
        'Score Separation': 'N/A',
        'Parameters': 'Majority voting'
    },
    {
        'Model': 'Ensemble (3 votes)',
        'Anomalies': ensemble_unanimous.sum(),
        'Percentage': f"{ensemble_unanimous.sum()/len(train_df)*100:.2f}%",
        'Score Separation': 'N/A',
        'Parameters': 'Unanimous'
    }
])

print("\nModel Comparison Summary:")
print(eval_summary.to_string(index=False))


MODEL EVALUATION AND COMPARISON

Model Comparison Summary:
              Model  Anomalies Percentage Score Separation                                                                               Parameters
   Isolation Forest        468      3.00%         0.129454 {'contamination': 0.03, 'n_estimators': 200, 'max_samples': 'auto', 'max_features': 1.0}
                LOF        468      3.00%         2.461622                                               {'contamination': 0.03, 'n_neighbors': 30}
             DBSCAN       1117      7.16%              N/A                                            {'eps': 1.1700134623709477, 'min_samples': 3}
Ensemble (≥2 votes)        225      1.44%              N/A                                                                          Majority voting
 Ensemble (3 votes)         16      0.10%              N/A                                                                                Unanimous


---

## 7. Generating Final Predictions

Creating a comprehensive results dataframe containing individual model predictions, ensemble votes, anomaly scores, and key features for each record.

**My Output Strategy:**
I'm building a comprehensive results dataset that includes:
- Individual model predictions (binary anomaly flags from each model)
- Individual model scores (continuous anomaly scores for ranking)
- Ensemble voting results (vote counts and voting threshold decisions)
- Ensemble anomaly score (composite score for prioritization)
- Key feature indicators (for quick anomaly characterization)

**Why I'm Including Multiple Outputs:**
- **Individual predictions**: Allow me to see which specific models flagged each record
- **Vote counts**: Show confidence level (1, 2, or 3 models agreeing)
- **Ensemble score**: Enables ranking of anomalies by severity
- **Key features**: Provide immediate context about what makes each record anomalous

**My Ranking Approach:**
I'm sorting the results by ensemble score (descending) so that:
- The most anomalous records appear first
- I can quickly identify top anomalies for investigation
- Stakeholders can review the most critical issues immediately

**Features I'm Including for Context:**
I'm adding key engineered features like:
- `is_critical_error`: Whether the record involves critical system errors
- `is_high_frequency`: Whether it represents unusually high activity
- `is_rare_operation`: Whether it involves uncommon operations
- `is_night_hours`: Whether it occurred during off-hours

These help me quickly understand the nature of detected anomalies.

In [19]:
print("\n" + "="*80)
print("GENERATING FINAL PREDICTIONS")
print("="*80)

# Creating comprehensive results DataFrame
results_df = pd.DataFrame({
    # Original record indices
    'record_index': range(len(train_df)),
    
    # Individual model predictions
    'IF_anomaly': if_anomalies.astype(int),
    'LOF_anomaly': lof_anomalies.astype(int),
    'DBSCAN_anomaly': dbscan_anomalies.astype(int),
    
    # Individual model scores
    'IF_score': if_scores,
    'LOF_score': lof_scores,
    
    # Ensemble results
    'ensemble_votes': ensemble_votes,
    'ensemble_score': ensemble_scores,
    'ensemble_majority': ensemble_majority.astype(int),
    'ensemble_unanimous': ensemble_unanimous.astype(int),
})

# Adding key features for context
results_df['is_critical_error'] = train_df['is_critical_error'].values
results_df['is_high_frequency'] = train_df['is_high_frequency'].values
results_df['is_rare_operation'] = train_df['is_rare_operation'].values
results_df['is_night_hours'] = train_df['is_night_hours'].values

print(f"* Created results dataframe: {results_df.shape}")

# Sorting by ensemble score (descending = most anomalous first)
results_df = results_df.sort_values('ensemble_score', ascending=False).reset_index(drop=True)

print(f"\nTop 10 most anomalous records:")
print(results_df[['record_index', 'ensemble_votes', 'ensemble_score', 
                   'is_critical_error', 'is_high_frequency', 'is_rare_operation', 'is_night_hours']].head(10))


GENERATING FINAL PREDICTIONS
* Created results dataframe: (15597, 14)

Top 10 most anomalous records:
   record_index  ensemble_votes  ensemble_score  is_critical_error  \
0          8064               2        0.825691                  0   
1         12061               2        0.501613                  1   
2          8302               3        0.491223                  1   
3          2193               2        0.489441                  1   
4          3209               2        0.487692                  1   
5          3293               2        0.486840                  1   
6          1759               2        0.485579                  1   
7          9384               2        0.483209                  1   
8         15013               2        0.476055                  1   
9          3703               2        0.475910                  1   

   is_high_frequency  is_rare_operation  is_night_hours  
0                  0                  0               0  
1         

---

## 8. Silhouette Score Analysis

Evaluating cluster quality using silhouette scores for both the ensemble model and pairwise model combinations. This metric measures how well-separated the anomalous and normal clusters are.

**Why I'm Using Silhouette Score:**
The silhouette score measures how similar a data point is to its own cluster compared to other clusters. It ranges from -1 to +1, where:
- **+1**: Perfect separation - points are well-matched to their cluster
- **0**: Points are on the border between clusters
- **-1**: Points might be assigned to the wrong cluster

I'm using this metric to validate that my anomaly detection models are creating meaningful separations between normal and anomalous records.

**My Analysis Strategy:**
I'm evaluating multiple configurations:
1. **Ensemble Model**: Using majority vote (≥2 models) as the clustering criterion
2. **Pairwise Combinations**: Testing all 2-model combinations (IF+LOF, IF+DBSCAN, LOF+DBSCAN)

This helps me understand:
- How well my full ensemble separates anomalies from normal data
- Which model pairs work best together
- Whether combining all three models improves or degrades separation quality

**My Computational Approach:**
Due to silhouette score's computational complexity (O(n²)), I'm using a representative sample of 5,000 records. This provides reliable estimates while maintaining reasonable computation time.

**My Interpretation of Ensemble Score:**

- **> 0.5 (EXCELLENT)**: My ensemble achieves strong separation - anomalies are clearly distinct from normal patterns. This gives me high confidence in the detections.

- **0.3-0.5 (GOOD)**: My ensemble achieves reasonable separation - most anomalies are distinguishable from normal data. This is acceptable for practical use.

- **0.1-0.3 (MODERATE)**: My ensemble shows weak but acceptable separation - there's some overlap between clusters. I might need to investigate borderline cases more carefully.

- **< 0.1 (POOR)**: My ensemble shows poor separation - significant overlap exists. This would suggest I need to revisit my feature engineering or model selection.


In [20]:
print("\n" + "="*80)
print("SILHOUETTE SCORE ANALYSIS - ENSEMBLE & PAIRWISE COMBINATIONS")
print("="*80)

# Using sample for computational efficiency
sample_size = min(5000, len(train_df))
sample_indices = np.random.RandomState(42).choice(len(train_df), size=sample_size, replace=False)
X_sample = train_df.iloc[sample_indices]
predictions_sample = results_df.iloc[sample_indices]

# ============================================================================
# PART A: ENSEMBLE SILHOUETTE SCORE (All 3 Models)
# ============================================================================
print("\n" + "-"*80)
print("PART A: ENSEMBLE MODEL (All 3 Models)")
print("-"*80)

print(f"\nCalculating Silhouette Score on sample of {sample_size:,} records...")

n_anomalies_in_sample = predictions_sample['ensemble_majority'].sum()

if n_anomalies_in_sample > 1 and n_anomalies_in_sample < len(predictions_sample) - 1:
    ensemble_sil_score = silhouette_score(X_sample, predictions_sample['ensemble_majority'])
    
    print(f"\n{'='*60}")
    print(f"* ENSEMBLE SILHOUETTE SCORE: {ensemble_sil_score:.4f}")
    print(f"{'='*60}")
    
    # Interpreting the silhouette score
    if ensemble_sil_score > 0.5:
        quality = "EXCELLENT"
        description = "Strong, well-separated clusters"
    elif ensemble_sil_score > 0.3:
        quality = "GOOD"
        description = "Reasonable cluster separation"
    elif ensemble_sil_score > 0.1:
        quality = "MODERATE"
        description = "Weak but acceptable separation"
    else:
        quality = "POOR"
        description = "Overlapping clusters, weak separation"
    
    print(f"\nInterpretation:")
    print(f"  Quality: {quality}")
    print(f"  Description: {description}")
    
    print(f"\nCluster Distribution in Sample:")
    print(f"  Normal records: {len(predictions_sample) - n_anomalies_in_sample:,} ({(1 - n_anomalies_in_sample/len(predictions_sample))*100:.1f}%)")
    print(f"  Anomalous records: {n_anomalies_in_sample:,} ({(n_anomalies_in_sample/len(predictions_sample))*100:.1f}%)")
else:
    ensemble_sil_score = None
    print("\n⚠ WARNING: Insufficient data for ensemble silhouette score")

# ============================================================================
# PART B: PAIRWISE MODEL COMBINATIONS
# ============================================================================
print("\n" + "="*80)
print("PART B: PAIRWISE MODEL COMBINATIONS (2 Models)")
print("="*80)

# Defining all pairwise combinations
model_combinations = [
    {
        'name': 'Isolation Forest + LOF',
        'models': ['IF', 'LOF'],
        'labels': (if_anomalies & lof_anomalies).astype(int)
    },
    {
        'name': 'Isolation Forest + DBSCAN',
        'models': ['IF', 'DBSCAN'],
        'labels': (if_anomalies & dbscan_anomalies).astype(int)
    },
    {
        'name': 'LOF + DBSCAN',
        'models': ['LOF', 'DBSCAN'],
        'labels': (lof_anomalies & dbscan_anomalies).astype(int)
    }
]

# Storing results for pairwise combinations
pairwise_results = []

print(f"\nEvaluating pairwise combinations on {sample_size:,} records...\n")
print("-" * 80)

for combo in model_combinations:
    combo_name = combo['name']
    combo_labels = combo['labels']
    
    # Getting labels for sample
    sample_labels = combo_labels[sample_indices]
    n_anomalies = sample_labels.sum()
    n_normal = len(sample_labels) - n_anomalies
    
    print(f"\n{combo_name}:")
    print(f"  Models: {' & '.join(combo['models'])}")
    print(f"  Anomalies (both agree): {n_anomalies:,} ({n_anomalies/len(sample_labels)*100:.2f}%)")
    
    # Calculating silhouette score if we have sufficient data
    if n_anomalies > 1 and n_normal > 1:
        sil_score = silhouette_score(X_sample, sample_labels)
        
        # Interpreting the score
        if sil_score > 0.5:
            quality = "EXCELLENT"
        elif sil_score > 0.3:
            quality = "GOOD"
        elif sil_score > 0.1:
            quality = "MODERATE"
        else:
            quality = "POOR"
        
        print(f"  Silhouette Score: {sil_score:.4f} ({quality})")
        
        pairwise_results.append({
            'Combination': combo_name,
            'Models': ' & '.join(combo['models']),
            'Anomalies': n_anomalies,
            'Anomaly %': f"{n_anomalies/len(sample_labels)*100:.2f}%",
            'Silhouette Score': sil_score,
            'Score_Str': f"{sil_score:.4f}",
            'Quality': quality
        })
    else:
        print(f"  ⚠ Insufficient data: {n_anomalies} anomalies, {n_normal} normal records")
        pairwise_results.append({
            'Combination': combo_name,
            'Models': ' & '.join(combo['models']),
            'Anomalies': n_anomalies,
            'Anomaly %': f"{n_anomalies/len(sample_labels)*100:.2f}%",
            'Silhouette Score': None,
            'Score_Str': 'N/A',
            'Quality': 'Insufficient data'
        })

# ============================================================================
# COMBINED SUMMARY
# ============================================================================
print("\n" + "="*80)
print("COMBINED SILHOUETTE SCORE SUMMARY")
print("="*80)

# Creating display dataframe
display_results = []
for result in pairwise_results:
    display_results.append({
        'Combination': result['Combination'],
        'Anomalies': result['Anomalies'],
        'Anomaly %': result['Anomaly %'],
        'Silhouette Score': result['Score_Str'],
        'Quality': result['Quality']
    })

# Adding ensemble result
if ensemble_sil_score is not None:
    if ensemble_sil_score > 0.5:
        ens_quality = "EXCELLENT"
    elif ensemble_sil_score > 0.3:
        ens_quality = "GOOD"
    elif ensemble_sil_score > 0.1:
        ens_quality = "MODERATE"
    else:
        ens_quality = "POOR"
    
    display_results.append({
        'Combination': 'Ensemble (All 3)',
        'Anomalies': n_anomalies_in_sample,
        'Anomaly %': f"{n_anomalies_in_sample/len(predictions_sample)*100:.2f}%",
        'Silhouette Score': f"{ensemble_sil_score:.4f}",
        'Quality': ens_quality
    })

summary_df = pd.DataFrame(display_results)
print("\n" + summary_df.to_string(index=False))

# Finding best performing combination
valid_scores = [r for r in pairwise_results if r['Silhouette Score'] is not None]
if valid_scores:
    best_combo = max(valid_scores, key=lambda x: x['Silhouette Score'])
    print(f"\n{'='*80}")
    print(f"BEST PERFORMING COMBINATION:")
    print(f"  {best_combo['Combination']}")
    print(f"  Silhouette Score: {best_combo['Score_Str']} ({best_combo['Quality']})")
    print(f"  Anomalies Detected: {best_combo['Anomalies']} ({best_combo['Anomaly %']})")
    print(f"{'='*80}")

print("\n" + "="*80)
print("Silhouette Score Guide:")
print("  > 0.5  : Excellent separation")
print("  0.3-0.5: Good separation")
print("  0.1-0.3: Moderate separation")
print("  < 0.1  : Poor separation")
print("="*80)

print("\n* Silhouette score analysis complete!")


SILHOUETTE SCORE ANALYSIS - ENSEMBLE & PAIRWISE COMBINATIONS

--------------------------------------------------------------------------------
PART A: ENSEMBLE MODEL (All 3 Models)
--------------------------------------------------------------------------------

Calculating Silhouette Score on sample of 5,000 records...

* ENSEMBLE SILHOUETTE SCORE: 0.1534

Interpretation:
  Quality: MODERATE
  Description: Weak but acceptable separation

Cluster Distribution in Sample:
  Normal records: 4,933 (98.7%)
  Anomalous records: 67 (1.3%)

PART B: PAIRWISE MODEL COMBINATIONS (2 Models)

Evaluating pairwise combinations on 5,000 records...

--------------------------------------------------------------------------------

Isolation Forest + LOF:
  Models: IF & LOF
  Anomalies (both agree): 6 (0.12%)
  Silhouette Score: 0.5616 (EXCELLENT)

Isolation Forest + DBSCAN:
  Models: IF & DBSCAN
  Anomalies (both agree): 19 (0.38%)
  Silhouette Score: 0.4936 (GOOD)

LOF + DBSCAN:
  Models: LOF & DBSCAN

---

## 9. Generating Anomaly Reports with Original Data

Merging the anomaly predictions with the original dataset to create comprehensive anomaly reports. Generating two outputs:
- **Majority Vote**: Anomalies flagged by at least 2 models (higher sensitivity)
- **Unanimous Vote**: Anomalies flagged by all 3 models (higher confidence)

**My Reporting Strategy:**
I'm creating two separate anomaly reports to support different use cases:

1. **Majority Vote Report (≥2 models)**:
   - **Purpose**: Comprehensive anomaly detection with balanced sensitivity
   - **Use case**: Regular monitoring, investigation queue, trend analysis
   - **Audience**: Data analysts, operations teams, routine investigations

2. **Unanimous Vote Report (3 models)**:
   - **Purpose**: High-confidence anomalies requiring immediate attention
   - **Use case**: Critical alerts, priority investigations, executive reporting
   - **Audience**: Security teams, management, incident response

**Why I'm Merging with Original Data:**
My model predictions contain only engineered features and anomaly scores. By merging with the original dataset, I'm adding:
- **Raw context**: Original timestamps, user IDs, operation types, etc.
- **Interpretability**: Business-relevant fields that explain what makes each record anomalous
- **Actionability**: Information needed for investigation and remediation

**My Data Integration Approach:**
1. Load original raw dataset from the source file
2. Map predictions back to original records using `original_index`
3. Merge predictions with original data for full context
4. Create separate filtered datasets for each voting strategy
5. Sort by ensemble score to prioritize most anomalous records

**Output Files I'm Creating:**
- `anomalies_majority_vote.csv`: Broader detection for comprehensive monitoring
- `anomalies_unanimous_vote.csv`: High-confidence alerts for priority action

In [21]:
print("\n" + "="*80)
print("GENERATING ANOMALY REPORTS WITH ORIGINAL DATA")
print("="*80)

# Creating output directory if it doesn't exist
import os
os.makedirs('results', exist_ok=True)
print("* Ensured results/ directory exists")

# Loading the original raw data
print("\nLoading original dataset...")
original_raw_data = pd.read_csv('data/anomaly_detection_assignment.csv')
print(f"* Original dataset shape: {original_raw_data.shape}")

# Getting the original_index from the features dataset
print("\nMapping predictions to original records...")
if 'original_index' in original_df.columns:
    results_df['original_index'] = original_df['original_index'].values
else:
    # Using sequential indices if no original_index exists
    results_df['original_index'] = range(len(results_df))
    print("  WARNING: No original_index found, using sequential indices")

# Merging predictions with original data
results_with_original = results_df.merge(
    original_raw_data,
    left_on='original_index',
    right_index=True,
    how='left'
)

print(f"* Merged results shape: {results_with_original.shape}")

# Generating Majority Vote Anomalies (≥2 votes)
print("\n" + "-"*80)
print("Generating Majority Vote Anomalies (≥2 models agree):")
print("-"*80)

majority_anomalies = results_with_original[results_with_original['ensemble_majority'] == 1].copy()
majority_anomalies = majority_anomalies.sort_values('ensemble_score', ascending=False)

print(f"* Total anomalies detected: {len(majority_anomalies):,}")
print(f"* Percentage of dataset: {len(majority_anomalies)/len(results_with_original)*100:.2f}%")

# Saving majority vote anomalies
majority_output_path = 'results/anomalies_majority_vote.csv'
majority_anomalies.to_csv(majority_output_path, index=False)
print(f"\n* Saved to: {majority_output_path}")

# Showing sample
print(f"\nTop 5 anomalies (Majority Vote):")
display_cols = ['original_index', 'ensemble_votes', 'ensemble_score', 'timestamp', 
                'user_id', 'operation_type', 'status_code', 'response_time_ms']
available_cols = [col for col in display_cols if col in majority_anomalies.columns]
print(majority_anomalies[available_cols].head(5).to_string(index=False))

# Generating Unanimous Vote Anomalies (3 votes)
print("\n" + "-"*80)
print("Generating Unanimous Vote Anomalies (all 3 models agree):")
print("-"*80)

unanimous_anomalies = results_with_original[results_with_original['ensemble_unanimous'] == 1].copy()
unanimous_anomalies = unanimous_anomalies.sort_values('ensemble_score', ascending=False)

print(f"* Total anomalies detected: {len(unanimous_anomalies):,}")
print(f"* Percentage of dataset: {len(unanimous_anomalies)/len(results_with_original)*100:.2f}%")

# Saving unanimous vote anomalies
unanimous_output_path = 'results/anomalies_unanimous_vote.csv'
unanimous_anomalies.to_csv(unanimous_output_path, index=False)
print(f"\n* Saved to: {unanimous_output_path}")

# Showing sample
print(f"\nTop 5 anomalies (Unanimous Vote):")
print(unanimous_anomalies[available_cols].head(5).to_string(index=False))

# Generating summary comparison
print("\n" + "="*80)
print("ANOMALY DETECTION SUMMARY")
print("="*80)

summary_data = {
    'Voting Strategy': ['Majority (≥2 models)', 'Unanimous (3 models)'],
    'Total Anomalies': [len(majority_anomalies), len(unanimous_anomalies)],
    'Percentage': [f"{len(majority_anomalies)/len(results_with_original)*100:.2f}%",
                   f"{len(unanimous_anomalies)/len(results_with_original)*100:.2f}%"],
    'Output File': ['anomalies_majority_vote.csv', 'anomalies_unanimous_vote.csv']
}

summary_df = pd.DataFrame(summary_data)
print("\n" + summary_df.to_string(index=False))

print(f"\n* All anomaly reports generated successfully!")
print(f"  Location: results/")


GENERATING ANOMALY REPORTS WITH ORIGINAL DATA
* Ensured results/ directory exists

Loading original dataset...
* Original dataset shape: (63713, 18)

Mapping predictions to original records...
* Merged results shape: (15597, 33)

--------------------------------------------------------------------------------
Generating Majority Vote Anomalies (≥2 models agree):
--------------------------------------------------------------------------------
* Total anomalies detected: 225
* Percentage of dataset: 1.44%

* Saved to: results/anomalies_majority_vote.csv

Top 5 anomalies (Majority Vote):
 original_index  ensemble_votes  ensemble_score
              0               2        0.825691
              1               2        0.501613
              2               3        0.491223
              3               2        0.489441
              4               2        0.487692

--------------------------------------------------------------------------------
Generating Unanimous Vote Anomalies (

---

## 10. Model Persistence and Deployment Preparation

Saving all trained models to disk for future use, deployment, and reproducibility.

**My Model Saving Strategy:**
I'm persisting all trained models using Python's pickle serialization format. This allows me to:
- **Reuse models**: Load trained models without retraining on new data
- **Deploy to production**: Package models for real-time anomaly detection systems
- **Version control**: Track model versions and performance over time
- **Reproduce results**: Ensure consistent predictions across different environments

**Models I'm Saving:**

1. **Isolation Forest** (`isolation_forest.pkl`):
   - My best-performing IF model with optimized hyperparameters
   - Contains trained trees and scoring logic
   - Can be used for fast anomaly scoring on new records

2. **LOF Model** (`lof_model.pkl`):
   - My tuned Local Outlier Factor model
   - Includes fitted neighbor structures for density comparison
   - Provides local density-based anomaly detection

3. **DBSCAN Model** (`dbscan_model.pkl`):
   - My optimized clustering-based outlier detector
   - Contains learned cluster structures
   - Enables spatial density-based anomaly identification

**Directory Structure I'm Creating:**
```
models/
├── isolation_forest.pkl    # Tree-based anomaly detection
├── lof_model.pkl          # Density-based local outlier detection
└── dbscan_model.pkl       # Clustering-based noise detection
```

**Deployment Considerations:**
- All models use the same feature set from feature engineering
- Models can be loaded independently or used as an ensemble
- Pickle format is Python-specific (for other languages, consider ONNX or PMML)
- Model files include all hyperparameters - no need to reconfigure

In [22]:
print("\n" + "="*80)
print("SAVING MODELS AND RESULTS")
print("="*80)

# Creating output directories if they don't exist
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)
print("* Ensured models/ and results/ directories exist")

# Save trained models
with open('models/isolation_forest.pkl', 'wb') as f:
    pickle.dump(best_if_model, f)
print("* Saved: models/isolation_forest.pkl")

with open('models/lof_model.pkl', 'wb') as f:
    pickle.dump(best_lof_model, f)
print("* Saved: models/lof_model.pkl")

if best_dbscan_model:
    with open('models/dbscan_model.pkl', 'wb') as f:
        pickle.dump(best_dbscan_model, f)
    print("* Saved: models/dbscan_model.pkl")


SAVING MODELS AND RESULTS
* Ensured models/ and results/ directories exist
* Saved: models/isolation_forest.pkl
* Saved: models/lof_model.pkl
* Saved: models/dbscan_model.pkl


<h1><center>END</center></h1>